In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import matplotlib
import math
import os
import numpy as np
from matplotlib.colors import LogNorm
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
import glob
import pandas as pd
import graphviz
from scipy import ndimage
from uncertainties import ufloat
from iminuit import Minuit
import scipy.optimize as opt
import scipy

In [ ]:
def sigmoid(x, x0, k):
    y = 1 / (1 + np.exp(-k * (x - x0)))
    return (y)


x_min = -0.5
x_max = 5
bin_numbers = 100
bin_lims = np.linspace(x_min, x_max , bin_numbers)
bin_centers = 0.5 * (bin_lims[:-1] + bin_lims[1:])

In [ ]:
def pars(x,a,b, c):
    return x*x*a+x*b+c
    
class Fit:
    def __init__(self, adcs, charge, curves, errs, p0x, p0ks):
        self.thresholds = np.array(adcs)
        self.x = np.array(charge)
        self.y = np.array(curves)
        self.y_err = np.array(errs)
        self.p0_a_x0 = p0x[0]
        self.p0_b_x0 = p0x[1]
        self.p0_a_k = p0ks[0]
        self.p0_b_k = p0ks[1]

    def start_fit(self):
        self.m = Minuit(self.all_LQs, a_x0=self.p0_a_x0, b_x0=self.p0_b_x0, a_k=self.p0_a_k, b_k=self.p0_b_k)
        self.m.migrad()
        
    def least_squares(self, model, measurement, error):
        return (model-measurement)**2/error**2

    def plot_fit(self,  a_x0, b_x0, c_x0, a_k, b_k, c_k):
        for adc, curve, curve_err in zip(self.thresholds, self.y, self.y_err):
            x0 = pars(adc, a_x0, b_x0,c_x0)
            k = pars(adc, a_k, b_k, c_k)
            plt.figure()
            plt.title(adc)
            model = sigmoid(self.x, x0, k)
            plt.plot(self.x, curve)
            plt.plot(self.x, model)
            plt.show()
            
    def all_LQs(self, a_x0, b_x0, c_x0, a_k, b_k, c_k):
        least_squares = 0

        for adc, curve, curve_err in zip(self.thresholds, self.y, self.y_err):
            x0 = pars(adc, a_x0, b_x0,c_x0)
            k = pars(adc, a_k, b_k, c_k)
            model = sigmoid(self.x, x0, k)
            least_squares+= np.sum(self.least_squares(model, curve, curve_err))

        return least_squares

In [ ]:
def SPE(q, 
        exp1_amp, exp1_width, 
        exp2_amp, exp2_width, 
        gaus_amp, gaus_mean, gaus_width):
    """
      The PDF for the SPE template using two exponentials + a gaussian
      
      Arguments:
          q: The charge values at which to evaluate the SPE template
          exp1_amp, exp1_width: The description of the first exponential distribution
          exp2_amp, exp2_width: The description of the second exponential distribution
          gaus_amp, gaus_mean, gaus_width: The parameters used for the gaussian distribution
          
      Returns:
          The SPE template pdf evaluated at q
    """
    with np.errstate(divide='ignore'):
        e=(q-gaus_mean)/(gaus_width)

        pdf = (  exp1_amp*np.exp(-q/exp1_width) 
               + exp2_amp*np.exp(-q/exp2_width) 
               + gaus_amp*np.exp(-0.5*e*e))
        pdf = np.nan_to_num(pdf)
        pdf /= (exp1_amp + exp2_amp + gaus_amp)
    return pdf
michael_fit = [6.89992979, 0.        , 0.5023212 , 0.20889681, 0.79379007, 1.        , 0.45267575]

def MPE(q,n,  
        exp1_amp, exp1_width, 
        exp2_amp, exp2_width, 
        gaus_amp, gaus_mean, gaus_width):

    # Grab the SPE template for these parameters
    pdf = SPE(q, 
              exp1_amp, exp1_width, 
              exp2_amp, exp2_width, 
              gaus_amp, gaus_mean, gaus_width)
    

    terms = []
    for i in [n-1]:
        with np.errstate(divide='ignore'):
            newpdf = pdf.copy()
            for _ in range(i):
                newpdf = scipy.signal.fftconvolve(pdf, newpdf, 'full')
            newpdf /= newpdf.sum() 
            terms.append(newpdf[:len(q)])
        
    # Merge all of the terms and return
    return np.array(terms).sum(axis=0)

def get_single_loss(passing_curve, charge_spectrum):
    after_trigger = passing_curve*charge_spectrum
    return scipy.integrate.simpson(after_trigger,x=XS)/scipy.integrate.simpson(charge_spectrum, x=XS)

XS = np.linspace(0.001,1*6,2000)
YS = SPE(XS, *michael_fit)
YS2 = MPE(XS,2, *michael_fit)
YS3 = MPE(XS,3, *michael_fit)

def get_all_losses(mean_charge, sigmoid_pars):
    xscaled=XS*mean_charge
    passing_curve = sigmoid(xscaled, *sigmoid_pars)
    losses = []
    for y in [YS, YS2, YS3]:
        losses.append(get_single_loss(passing_curve, y) )
    return losses


In [ ]:
#In this data file the histograms of charge spectra with and without discriminator flag were stored setting a 2mV discriminator level above noise

data = np.loadtxt("241014_2mV_calibration_chargeblock.txt" ,unpack=1)
channel, disc_level = data[0:2] 
rest = data[2:].T

full = []
disc = []
for line in rest:
    full.append(line[:int(rest[0].size/2)])
    disc.append(line[int(rest[0].size/2):])

print(rest.shape, channel.size, rest[0].size, disc[0].size)

In [ ]:
# Fit sigmoids to all the data
for PMT in range(24):
    mask_channel = channel == PMT
    thresholds = []

    popts0 = []
    popts1 = []
    pcovs = []
    
    for i, threshold in enumerate(disc_level):
        if mask_channel[i]:
            thresholds.append(threshold)
            mask = bin_centers>0.1
            mask = np.logical_and(mask, bin_centers<2)
            
            full_i = full[i]
            disc_i = disc[i]
            fu = [ufloat(disc_i[j], np.sqrt(disc_i[j]+1))/ufloat(full_i[j], np.sqrt(full_i[j])) if full_i[j]>0 else ufloat(0,np.inf) for j in range(full_i.size) ]
            f = np.array([n.n for n in fu])
            err = np.array([n.s for n in fu])
    
            p0 = [0.2, 1.2]

            
            popt, pcov = curve_fit(sigmoid, bin_centers[mask], f[mask], p0=p0, sigma=err[mask], absolute_sigma=1)
            print(PMT, popt)
            popts0.append(popt[0])
            popts1.append(popt[1])
            pcovs.append(pcov)
    np.savetxt(f"sigmoid_fits/241014_threshold_to_sigmoid_{PMT}", [thresholds, popts0, popts1])
    np.save(f"sigmoid_fits/241014_threshold_to_sigmoid_pcovs_{PMT}", pcovs)

In [ ]:
fname = "/HDD/backuped/Promotion_data/Postdoc/240822_mDOM_efficiency/pre_analysis/output_data/241014_gain_fits.dat"
theta, pmt, phi, wavelength, mcharge, charge_err, width, widht_err, gof = np.loadtxt(fname, unpack=1)
with open(f"/HDD/backuped/Promotion_data/Postdoc/240822_mDOM_efficiency/pre_analysis/output_data/241014_mean_gain.dat", "w") as file:
    pass
phis = np.unique(phi)
plt.figure()
for PMT in range(24):

    mean = []
    errors = []
    for up in phis:
        
        mask = np.logical_and(pmt==PMT, phi ==up)
        mask = np.logical_and(mask, ~np.isnan(mcharge ))
        mean.append(np.average(mcharge[mask], weights=1/charge_err[mask]**2))
        errors.append(np.std(mcharge[mask])/np.sqrt(mcharge[mask].size-1))
    mean = np.array(mean)
    plt.errorbar(phis, mean/np.mean(mean), yerr = errors/np.mean(mean), fmt = ".")
    with open(f"/HDD/backuped/Promotion_data/Postdoc/240822_mDOM_efficiency/pre_analysis/output_data/241014_mean_gain.dat", "a") as file:
        for pp, mm, ee in zip(phis, mean, errors):
            for val in [PMT, pp,mm,ee]:
                file.write(f"{val}\t")
            file.write("\n")
plt.show()



In [ ]:
fname = "/HDD/backuped/Promotion_data/Postdoc/240822_mDOM_efficiency/pre_analysis/output_data/241014_mean_gain.dat"
PMTs, phi, gain, gain_error = np.loadtxt(fname, unpack=1)
with open("241014_loss_per_phi.dat", "w") as f:
    pass
for PMT in range(24):

    mask = PMTs==PMT
    t,x0s, ks =  np.loadtxt(f"sigmoid_fits/241014_threshold_to_sigmoid_{PMT}")
    
    loss = []
    error = []
    for mean_charge, mean_charge_err in zip(gain[mask], gain_error[mask]):
        losses = []
        for ch in np.random.normal(mean_charge, mean_charge_err, 3):
            for x0, k in zip(x0s, ks):
                losses.append(get_all_losses(mean_charge, [x0, k]))
        loss.append(np.mean(losses, axis=0))
        error.append(np.std(losses,axis=0)/np.sqrt(len(losses)-1))

    with open("241014_loss_per_phi.dat", "a") as f:
        for pp, loss_triplet, error_triplet in zip(phi[mask], loss, error):
            for loss_i, error_i in zip(loss_triplet, error_triplet):
                if pp==255.94:
                    print(loss_triplet)
                f.write(f"{PMT}\t{pp}\t{loss_i:.5g}\t{error_i:.5g}\n")
    plt.figure()
    for i in [0]:
        plt.errorbar(phi[mask], [n[i] for n in loss], yerr= [n[i] for n in error], fmt = ".")
    plt.title(PMT)
    plt.grid()
    plt.show()
    

In [ ]:
class PassingTriggerData:
    def __init__(self, fname: str):
        PMT, phi, passing_fraction, error = np.loadtxt(fname, unpack=True)
        self._data_dict = {}
        for pmt, phi, passing_fraction, error in zip(PMT, phi, passing_fraction, error):
            if (pmt, phi) not in self._data_dict:
                self._data_dict[(pmt, phi)] = ([], [])
            self._data_dict[(pmt, phi)][0].append(passing_fraction)
            self._data_dict[(pmt, phi)][1].append(error)
        self._sort_data()
        
    def _sort_data(self):
        for key in self._data_dict:
            fractions, errors = self._data_dict[key]
            argsort = np.argsort(fractions)
            self._data_dict[key] = ([fractions[i] for i in argsort], [errors[i] for i in argsort])

    def get_trigger_loss(self, phi: int, PMT: int):
        return self._data_dict[(PMT, phi)]

In [ ]:
passing = PassingTriggerData("241014_loss_per_phi.dat")

In [ ]:
passing.get_trigger_loss(2.81, 0)

# For geant

In [ ]:
def sigmoid(x, x0, k):
    y = 1 / (1 + np.exp(-k * (x - x0)))
    return (y)
    
popts = []
ch = np.linspace(0,3,1000)
for PMT in range(24):
    t, x0s, ks=  np.loadtxt(f"sigmoid_fits/241014_threshold_to_sigmoid_{PMT}")
    print(t.size)
    pcovs =  np.load(f"sigmoid_fits/241014_threshold_to_sigmoid_pcovs_{PMT}.npy")
    curves = []
    for x0, k, cov_matrix in zip(x0s, ks, pcovs):
        L = np.linalg.cholesky(cov_matrix)
        popt = [x0, k]
        for _ in range(10):
            # Generate correlated random samples
            param_samples = popt + L @ np.random.randn(len(popt))
            curves.append(sigmoid(ch, *param_samples))
    plt.figure() 
    for curve in curves:
        plt.plot(ch, curve, alpha = 0.005, color = "black")
    p0 = [0.2, 1.2]
    popt, pcov = curve_fit(sigmoid, ch, np.mean(curves, axis=0), p0=p0)
    popts.append(popt)
    plt.plot(ch, np.mean(curves, axis=0))
    plt.plot(ch, sigmoid(ch, *popt), "--")
    plt.title(PMT)
    plt.show()
    
    

In [ ]:
PMT2AFE = {
    0: 6,
    1: 14,
    2: 20,
    3: 2,
    4: 8,
    5: 10,
    6: 12,
    7: 16,
    8: 18,
    9: 22,
    10: 0,
    11: 4,
    12: 7,
    13: 11,
    14: 13,
    15: 17,
    16: 19,
    17: 23,
    18: 1,
    19: 5,
    20: 3,
    21: 9,
    22: 15,
    23: 21
}

AFE2PMT = {}

for key, val in PMT2AFE.items():
    AFE2PMT[val] = key
with open("geant_mean_sigmoid.txt", "w") as f:
    for PMT, popt in enumerate(popts):
        f.write(f"{AFE2PMT[PMT]} {popt[0]} {popt[1]}\n")
        

In [ ]:
fname = "/HDD/backuped/Promotion_data/Postdoc/240822_mDOM_efficiency/pre_analysis/output_data/241014_mean_gain.dat"
PMTs, phis, gains, gain_error = np.loadtxt(fname, unpack=1)

with open("geant_mean_gain.txt", "w") as f:
    for PMT, phi, gain in zip(PMTs, phis, gains):
        f.write(f"{AFE2PMT[PMT]} {phi} {gain}\n")